In [ ]:
IS_OSC = False

In [ ]:
# Setup script to run on OSC
# conda install scikit-learn matplotlib nltk seaborn
# pip install git+https://github.com/titipata/pubmed_parser.git@0.3.1
# conda install -c conda-forge hummingbird-ml
# conda install ipykernel
# ipython kernel install --user --name=thesis


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.pipeline import Pipeline
import seaborn as sns
import pubmed_parser as pp
import csv
from tqdm import tqdm
from hummingbird.ml import convert

import warnings
warnings.filterwarnings('ignore')

Let's look at one pub-med dictionary entry:

In [ ]:
if IS_OSC:
    data_path = "/ftp.ncbi.nlm.nih.gov/pubmed/baseline/"
else:
    data_path = '../../data/raw'
path_xml = pp.list_xml_path(data_path) # list all xml paths under directory
pubmed_dict = pp.parse_medline_xml(path_xml[0]) # dictionary output

Load it into a dataframe and output the head

In [ ]:
df = pd.DataFrame(pubmed_dict)
df.head()

Drop the missing abstracts

In [ ]:
df = df[df['abstract'] != '']
df.shape

Get the unique mesh terms

In [ ]:
mesh_terms_arr = []
[mesh_terms_arr.extend(mesh_terms.split('; ')) for mesh_terms in df.mesh_terms.to_list()]
unique_mesh_terms = set(mesh_terms_arr)

# Display the first 20 unique terms
list(unique_mesh_terms)[:20]

Add the mesh terms to the dataframe as columns. This does take about 5 minutes to run per volume.

In [ ]:
for mesh_term in tqdm(unique_mesh_terms):
    df[mesh_term] = df.mesh_terms.apply(lambda x: 1 if mesh_term in x else 0)

Display the head of the dataframe to see if the mesh terms were added

In [ ]:
df.head()

Get the counts of the mesh terms

In [ ]:
mesh_term_counts = []
for mesh_term in tqdm(unique_mesh_terms):
    mesh_term_counts.append((mesh_term, df[mesh_term].sum()))
mesh_term_counts = sorted(mesh_term_counts, key=lambda x: x[1], reverse=True)
mesh_term_df = pd.DataFrame(mesh_term_counts, columns=['mesh_term', 'count'])
mesh_term_df.head(20)

Plot the top 20 mesh terms

In [ ]:
mesh_term_df.head(20).plot.bar(x='mesh_term', y='count', figsize=(20, 10))

How many mesh terms are there per article?

In [ ]:

count_of_mesh_terms_per_article = df.iloc[:,22:].sum(axis=1).value_counts()


In [ ]:
count_of_mesh_terms_per_article.plot.bar(figsize=(20, 10))

Let's start training and testing

In [ ]:
train, test = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)

In [ ]:
X_train = train.abstract + ' ' + train.title
X_test = test.abstract + ' ' + test.title
print(X_train.shape)
print(X_test.shape)

Create a pipeline with a multi label classifier

In [ ]:
def classifier_effectiveness(name, pipeline):
    with open(name + '-accuracy.csv','w') as f1:
        writer=csv.writer(f1, delimiter=',',lineterminator='\n')
        writer.writerow(['mesh_term','term_count','prediction_count','accuracy','precision','recall'])
        for mesh_term in tqdm(unique_mesh_terms):
            # train the model using X_dtm & y
            pipeline.fit(X_train, train[mesh_term])
            # compute the testing accuracy
            prediction = pipeline.predict(X_test)
            if prediction.sum() > 0:
                print('mesh_term: ', mesh_term)
                print(test[mesh_term].sum())
                print(prediction.sum())
                print('Test accuracy is {}'.format(accuracy_score(test[mesh_term], prediction)))
                print('Test precision is {}'.format(precision_score(test[mesh_term], prediction)))
                print('Test recall is {}'.format(recall_score(test[mesh_term], prediction)))


            writer.writerow([
                mesh_term, 
                test[mesh_term].sum(),
                prediction.sum(),
                accuracy_score(test[mesh_term], prediction), 
                precision_score(test[mesh_term], prediction), 
                recall_score(test[mesh_term], prediction)
                ])

In [ ]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
classifier_effectiveness('nb', NB_pipeline)

In [ ]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
classifier_effectiveness('svc', SVC_pipeline)

In [ ]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
classifier_effectiveness('logreg', LogReg_pipeline)